# INSTALL AND IMPORT DEPENDENCIES

In [ ]:
%pip install matplotlib
%pip install easyocr
%pip install imutils
%pip install numpy
%pip install pyqt5

In [ ]:
import cv2
import imutils
import easyocr
import numpy as np
from matplotlib import pyplot as plt

# (OPTIONAL) SELECT IMAGE TO PROCESS

In [ ]:
from PyQt5.QtWidgets import QApplication, QFileDialog

# Create a QApplication instance
app = QApplication([])

# Open the file dialog and get the selected file path
file_dialog = QFileDialog()
file_path, _ = file_dialog.getOpenFileName()

# Print the selected file path
print("Selected file:", file_path)

# Close the application
app.quit()

# READ AN IMAGE, GRAYSCALE AND BLUR

## IF NO IMAGE WAS SELECTED, THEN WRITE THE PATH OF THE IMAGE

In [ ]:
img = cv2.imread(file_path)

In [ ]:
print("IMAGEN A DETECTAR")
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))


In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
print("IMAGEN EN ESCALA DE GRISES")
plt.imshow(cv2.cvtColor(gray, cv2.COLOR_BGR2RGB))


In [ ]:
gray = cv2.blur(gray, (3, 3))

In [ ]:
print("IMAGEN EN ESCALA DE GRISES CON BLUR")
plt.imshow(gray, cmap="gray")

# APPLY FILTER AND FIND EDGES FOR LOCALIZATION

In [ ]:
bfilter = cv2.bilateralFilter(gray, 11, 17, 17)  # Noise reduction
edged = cv2.Canny(bfilter, 30, 200)  # Edge detection

In [ ]:
print("IMAGEN CON BORDES DETECTADOS")
plt.imshow(cv2.cvtColor(edged, cv2.COLOR_BGR2RGB));


# FIND CONTOURS AND APPLY MASK

In [ ]:
keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(keypoints)
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

In [ ]:
location = None
for contour in contours:
    approx = cv2.approxPolyDP(contour, 10, True)
    if len(approx) == 4:
        location = approx
        break

In [ ]:
print(f"POSIBLE UBICACION DE LA PLACA\n{location}")

In [ ]:
mask = np.zeros(gray.shape, np.uint8)
new_image = cv2.drawContours(mask, [location], 0, 255, -1)
new_image = cv2.bitwise_and(img, img, mask=mask)

In [ ]:
print("IMAGEN CON LA PLACA DETECTADA")
plt.imshow(cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB))


In [ ]:
(x, y) = np.where(mask == 255)
(topx, topy) = (np.min(x), np.min(y))
(bottomx, bottomy) = (np.max(x), np.max(y))
cropped = gray[topx : bottomx + 1, topy : bottomy + 1]

In [ ]:
print("IMAGEN DE LA PLACA RECORTADA")
plt.imshow(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))


# USE EASY OCR TO READ TEXT

In [ ]:
reader = easyocr.Reader(["en"])
result = reader.readtext(cropped)

In [ ]:
print("TEXTO DETECTADO EN LA PLACA")
print(result)

# RENDER RESULT

In [ ]:
text = result[0][-2]
font = cv2.FONT_HERSHEY_SIMPLEX
res = cv2.putText(
    img,
    text=text,
    org=(approx[0][0][0], approx[1][0][1] + 60),
    fontFace=font,
    fontScale=1,
    color=(0, 255, 0),
    thickness=2,
    lineType=cv2.LINE_AA,
)
res = cv2.rectangle(img, tuple(approx[0][0]), tuple(approx[2][0]), (0, 255, 0), 3)


In [ ]:
print("IMAGEN CON EL TEXTO DE LA PLACA DETECTADO")
plt.imshow(cv2.cvtColor(res, cv2.COLOR_BGR2RGB))
